In [63]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re


In [64]:

def split_numbers_and_street(string):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string) if s]

## Reading from a local file

In [68]:

def scrape_madlan():
	page = 1
	data = []
	while page<= 1:
		with open('h1.html', 'r', encoding='utf-8') as f:
			response = f.read()
		soup = BeautifulSoup(response, 'html.parser')

		apartments = soup.find_all('div',{'data-auto': 'property-details'} )
		print(apartments)
		if not apartments:
			print('er')
			break
		for apartment in apartments:
			price_elem = apartment.find('div', {'data-auto': 'property-price'})
			price = price_elem.text.strip() if price_elem else None
			room_number_elem = apartment.find('div', {'data-auto': 'property-rooms'})
			room_number = room_number_elem.text.strip() if room_number_elem else None
			area_elem = apartment.find('div', {'data-auto': 'property-size'})
			area = area_elem.text.strip() if area_elem else None
			apt_type_elem = apartment.find('div', {'data-auto': 'property-class'})
			apt_type = apt_type_elem.text.strip() if apt_type_elem else None
			address_elem = apartment.find('div', {'data-auto': 'property-address'})
			address = address_elem.text.strip().split(',') if address_elem else None
			street = None
			number_in_street = None
			city_area = None
			if  address:
				street_num = split_numbers_and_street(address[0])
				if len(street_num)==2:
					number_in_street = street_num[1]
					street = street_num[0]
				else:
					street= street_num[0]
				if len(address)==2:
					city_area = address[1].strip()
				else:
					city_area = None
			data.append([apt_type, room_number, area, street, number_in_street, city_area, price])
		print(page)
		page += 1
		
	df = pd.DataFrame(data, columns=['type', 'room_number', 'Area', 'Street', 'number_in_street', 'city_area', 'price'])
	return df
df = scrape_madlan()


[<div class="css-l57rop eu6esji2" data-auto="property-details"><style data-emotion-css="dywgfi">.css-dywgfi{-webkit-letter-spacing:normal;-moz-letter-spacing:normal;-ms-letter-spacing:normal;letter-spacing:normal;color:inherit;font-family:NovemberHebrew;font-weight:700;font-size:17px;line-height:24px;color:#303030;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-box-pack:justify;-webkit-justify-content:space-between;-ms-flex-pack:justify;justify-content:space-between;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;}</style><div class="css-dywgfi eu6esji1"><style data-emotion-css="19tpcfu">.css-19tpcfu{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;}</style><div class="css-19tpcfu eu6esji3" data-auto="property-price">₪1,030,000</div></div><style data-emotion-css="1yo09r9">@media screen and (m

In [67]:

def scrape_madlan():
    base_url = "https://www.madlan.co.il/for-sale/נוף-הגליל-ישראל"
    page = 1
    max_page = 3
    data = []
    while page<= max_page:
        url = f"{base_url}?page={page}"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
        response = requests.get(url,headers= headers)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        if response.status_code == 200:
            apartments = soup.find_all('div',{'data-auto': 'property-details'} )
            print(apartments)
            if not apartments:
                print('er')
                break
            for apartment in apartments:
                price_elem = apartment.find('div', {'data-auto': 'property-price'})
                price = price_elem.text.strip() if price_elem else None
                room_number_elem = apartment.find('div', {'data-auto': 'property-rooms'})
                room_number = room_number_elem.text.strip() if room_number_elem else None
                area_elem = apartment.find('div', {'data-auto': 'property-size'})
                area = area_elem.text.strip() if area_elem else None
                apt_type_elem = apartment.find('div', {'data-auto': 'property-class'})
                apt_type = apt_type_elem.text.strip() if apt_type_elem else None
                address_elem = apartment.find('div', {'data-auto': 'property-address'})
                address = address_elem.text.strip().split(',') if address_elem else None
                street = None
                number_in_street = None
                city_area = None
                if  address:
                    street_num = split_numbers_and_street(address[0])
                    if len(street_num)==2:
                        number_in_street = street_num[1]
                        street = street_num[0]
                    else:
                        street= street_num[0]
                    if len(address)==2:
                        city_area = address[1].strip()
                    else:
                        city_area = None
                data.append([apt_type, room_number, area, street, number_in_street, city_area, price])
            print(page)
            page += 1
        else:
            print (f' erorr status code {response.status_code}')
            break
    df = pd.DataFrame(data, columns=['type', 'room_number', 'Area', 'Street', 'number_in_street', 'city_area', 'price'])
    return df
df = scrape_madlan()


 erorr status code 403


In [69]:
df['number_in_street'] = pd.to_numeric(df['number_in_street'], errors='coerce').astype('Int64')
df['CITY'] = 'נוף הגליל'
df = df.dropna(subset= 'price').reset_index(drop=True)


In [70]:
df

,type,room_number,Area,Street,number_in_street,city_area,price,CITY
0,דירה,3 חד׳,"70 מ""ר",דרך אריאל שרון,19,שלום,"₪1,030,000",נוף הגליל
1,דירה,4 חד׳,"91 מ""ר",הרב עובדיה יוסף,1,צפונית,"₪1,050,000",נוף הגליל
2,קוטג',6 חד׳,"151 מ""ר",ירדן,<NA>,זאב,"₪2,580,000מתיווך",נוף הגליל
3,דירה,2 חד׳,"45 מ""ר",מנחם אריאב,27,צפונית,"₪660,000",נוף הגליל
4,דירה,4 חד׳,"106 מ""ר",חרוד,16,אשכול,"₪1,230,000",נוף הגליל
5,דירה,4 חד׳,"106 מ""ר",חרוד,16,אשכול,"₪1,350,000",נוף הגליל
6,דירה,4 חד׳,"100 מ""ר",אלומה,6,הר יונה,"₪1,530,000",נוף הגליל
7,דירה,3 חד׳,"74 מ""ר",דרך אריאל שרון,19,שלום,"₪950,000",נוף הגליל
8,דירה,4 חד׳,"100 מ""ר",הרב פנחס מילר,4,הר יונה ג',"₪1,220,000",נוף הגליל
9,דירה,3 חד׳,"66 מ""ר",דרך אריאל שרון,21,שלום,"₪850,000",נוף הגליל


In [ ]:
df.to_excel('result.xlsx')